# askmydoc colab

[see it in action](https://askmydoc.app/)

## Setup

In [ ]:
!zip -r /content/file.zip /content/askmydoc-workshop

In [ ]:
!git clone https://github.com/UriKatsirPrivate/askmydoc-colab.git

Cloning into 'askmydoc-colab'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 29 (delta 6), reused 28 (delta 5), pack-reused 0
Receiving objects: 100% (29/29), 807.71 KiB | 14.69 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [ ]:
!pip install google-cloud-aiplatform
!pip install langchain chromadb watchdog pypdf bs4 tabulate
!pip install streamlit

In [ ]:
# Restart the runtime
# import os
# os.kill(os.getpid(), 9)

In [ ]:
from google.colab import auth as google_auth
google_auth.authenticate_user()

In [ ]:
# %%writefile app_sample.py

# import streamlit as st

# st.write('Hello, *World1* :sunglasses:')



Writing app_sample.py


In [ ]:
!npm install -q localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 2.018s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



In [ ]:
!streamlit run /content/askmydoc-colab/app.py &>/content/logs.txt & curl ipv4.icanhazip.com & npx localtunnel --port 8501

35.197.6.31
npx: installed 22 in 2.832s
your url is: https://orange-cats-exist.loca.lt
^C


## Running Shell Commands

[examples](https://colab.sandbox.google.com/drive/1N7p0B-7QWEQ9TIWRgYLueW03uJgJLmka#scrollTo=i7cDqnvavT9i) and [here](https://colab.sandbox.google.com/github/PlantsAndPython/PlantsAndPython/blob/master/M_7_SCRIPTING_WITH_BASH/0_Lessons/7.1_Scripting_with_bash.ipynb)

In [ ]:
%%shell

export GOOGLE_CLOUD_PROJECT=landing-zone-demo-341118



In [ ]:
%%shell

cd askmydoc-colab
chmod +x deploy.sh
sh deploy.sh


Updated property [core/project].
Creating temporary tarball archive of 16 file(s) totalling 21.1 KiB before compression.
Uploading tarball of [.] to [gs://landing-zone-demo-341118_cloudbuild/source/1695247002.028676-7b876b067be24326900ef64a58baadb1.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/landing-zone-demo-341118/locations/global/builds/151cfebc-88b4-41a2-9140-7deaeb7de484].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/151cfebc-88b4-41a2-9140-7deaeb7de484?project=854735162550 ].
 REMOTE BUILD OUTPUT
starting build "151cfebc-88b4-41a2-9140-7deaeb7de484"

FETCHSOURCE
Fetching storage object: gs://landing-zone-demo-341118_cloudbuild/source/1695247002.028676-7b876b067be24326900ef64a58baadb1.tgz#1695247002580453
Copying gs://landing-zone-demo-341118_cloudbuild/source/1695247002.028676-7b876b067be24326900ef64a58baadb1.tgz#1695247002580453...
/ [1 files][  6.9 KiB/  6.9 KiB]                                                
Operation completed o